In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class GraphState(TypedDict):
	messages : Annotated[list, add_messages] 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o-mini',temperature=0)

def chatbot(state : GraphState):
    return {"messages" : [llm.invoke(state['messages'])]}

# 빈 껍데기 생성
graph_builder = StateGraph(GraphState)

# 추가
graph_builder.add_node("chatbot", chatbot)

# 시작
graph_builder.add_edge(START, "chatbot")

# 끝
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()

question = "서울의 유명한 맛집 TOP 5 추천해줘"

# 그래프 이벤트 스트리밍
for event in graph.stream({"messages": [("user", question)]}):
    # 이벤트 값 출력
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)